# Prétraitement, codage et reconnaissance du locuteur

In [1]:
import numpy as np
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio,display

# Les pré-requis

Fonctions de conversion utiles

In [2]:
#convertir une fréquence en mel vers une fréquence en Hz
def Mel2Hz(mel):
    return 700 * (np.power(10,mel/2595)-1)

#convertir une fréquence en Hz vers une fréquence en Mel
def Hz2Mel(freq):
    return 2595 * np.log10(1+freq/700)

#convertir une fréquence Hz en indice de matrice (sachant que fs --> T)
def Freq2Ind(freq,fs,T):
    return (freq*T/fs).astype(int)

#Fenêtre de hamming
def hamming(T):
    t=np.arange(T)
    return 0.54-0.46*np.cos(2*np.pi*t/(T-1))

# Partie prétraitement : 

Les fonctions qu'on vas utilisées pour le prétraitement de la parole (filtrage du bruit, fenetrage de hamming, calcul de spectrogram)

In [3]:
#Filtre de préaccentuation - Augmenter les hautes fréquences
def preaccentuation(x):
    return x[:-1]-0.97*x[1:]


#Fenêtre de hamming - Pondération des fenêtres
def hamming(T):
    t=np.arange(T)
    return 0.54-0.46*np.cos(2*np.pi*t/(T-1))


#calcul du spectrogram
def spectrogram(x,T,p,window=None,fftsz=None):
    n=len(x) #taille de x
    m=len(np.arange(0,n-T,p)) #estimation du nombre de fenêtres
    S=np.zeros((m,T))

    if fftsz is None: fftsz=T #si vide on prend la valeure de T
    if window is None: window=np.ones(T) #si vide on prend des 1
    
    #début fenêtrage - découpage du signal
    for i in range(m):
        S[i,:]=x[i*p:i*p+T]*window
            
    #Transformée de Fourier - Passage du domaine temporel vers le domaine spectral        
    S=np.fft.fft(S,fftsz)
    
    amp=np.abs(S) #spectre d'amplitude
    phase=np.angle(S) #spectre de phase
    
    return amp,phase


# Partie Codage : 

Extraction des caractéristiques (création des filtres oreille pour ensuite pouvoir extraire les paramètres pertinentes MFCC)

In [21]:
#Cette fonction va s'en charger de créer les filtres oreilles  
#Créer des filtres proches de ceux de l'oreille, nf: nombre de filtres à créer
def filtresOreille(nf,T,fs):
    freqMin=50  #Hz On définit la fréquence minimum que perçoit l'oreille
    freqMax=min(10000, fs/2)  #Hz On définit la fréquence maximal que perçoit l'oreille 

    #convertir en mel
    melMin=Hz2Mel(freqMin) # 50Hz = 77mel
    melMax=Hz2Mel(freqMax) # 10000Hz = 3073mel

    #calcul du début et la fin de chaque filtre en Mel
    pas=(melMax-melMin)/(nf+1)  #diviser l'espace de perception de l'oreille sur le nombre de filtres
    debut=np.arange(melMin,melMax-2*pas+1,pas) #réalisation de filtres qui se chevauchent de moitier
    fin=debut+2*pas
    
    #convertir les filtres dans les fréquences (Hz)
    debuthz=Mel2Hz(debut)
    finhz=Mel2Hz(fin)

    #convertir les fréquences en indices
    debutI=Freq2Ind(debuthz,fs,T)  
    finI=Freq2Ind(finhz,fs,T)

    #construie les filtres 
    filtres=np.zeros((int(T/2),nf))
    for i in range(len(debutI)):
        filtres[debutI[i]:finI[i],i]=hamming(finI[i]-debutI[i])
    
    return filtres

La fonction mfcc tiré de la dernière partie de la demonstration des MFCC

In [18]:
# extraction des 13 paramètres pertinentes
#Calcul des coefficients MFCC, data: le vecteur des données (domaine temporel), T: taille de la fenêtre 
# p: le pas , filtres: matrice des filtres de l'oreille, nc: le nombre de coefficients à garder
def mfcc(data,filtres,T=512,p=256,nc=13):
    amp,phase=spectrogram(preaccentuation(data),T,p,window=hamming(T))  #calcul du spectrogramme
    amp=amp[:,:int(T/2)] #On coupe le spectre d'amplitude en 2 à cause de l'effet miroir
    amp=np.dot(amp,filtres) #application des filtres de l'oreille 
    amp=np.log(amp) #calcul du log du spectre (convertir la multiplication en addition)
    amp=idct(amp, norm = 'ortho') #calcul de l'idct du spectre filtré en log
    amp=amp[:,:nc] #ne choisir que les "nc" premiers (réduction des données + lissage ceptral)
    
    return amp

# Partie Reconnaissance

Debut du travail avec le Chargement du Dataset, ensuite arrive la Partie prétraitement et codage detaillé en haut.

In [23]:
######################Chargement du Dataset##############################################################
chemin='C:\\Users\\pc\Desktop\\Mes Documents\\Eddirassa\\MI M1\\S2\Mes TPs\\TP TParoles\\Ressources\\free-spoken-digit-dataset-master\\recordings\\'
locuteurs=['jackson','nicolas','theo','yweweler']


datamfcc=[] #on va stocker les mfcc de chaque fichier audio dans cette liste
labels=[] #on va stocker la classe réelle [0-9] de chaque fichier audio dans cette liste


# Lecture, Normalisation et extraction des mfcc pour chaque chiffre des quatres locuteurs
for ch in range(10):  #pour les chiffres de 0-9
    for loc in locuteurs: #pour chaque locuteur
        for i in range(50): #chaque locuteur a répété 50 fois le même chiffre
            fichier="%d_%s_%d.wav"%(ch,loc,i) #le nom du fichier à charger
            
            fs, data = wavfile.read(chemin+fichier) #lire un fichier audio 
            data=data/max(abs(data))  #le normaliser entre -1 et 1
            T=512
            n=256
            nf=36
            nc=13
            filtres=filtresOreille(nf,T,fs) #construction des filtres
            fichier_mfcc=mfcc(data,filtres) #calculer les mfccs
            #Donc les parametres mfcc extrais de tt les audions sont stocké dans fichier_mfcc
          
            
            datamfcc.append(fichier_mfcc) #ajouter les mfcc du fichier à la liste
            labels.append(ch) #ajouter la classe du fichier aux labels (s'il s'agit d'un 0 ou 1 ou 2...)
            

In [33]:
#mélange aléatoire du dataset et division 50% train / 50% test
n=len(labels)  #le nombre de fichiers

indices=np.arange(n)  #mélange aléatoire du dataset
np.random.shuffle(indices)
datamfcc=np.array(datamfcc)[indices]
labels=np.array(labels)[indices]

m=int(n*0.5)  #prendre 50% comme train et 50% comme test
trainmfcc=datamfcc[:m] #données d'apprentissage
trainlabels=labels[:m] #sortie désirée pour chaque donnée d'apprentissage
testmfcc=datamfcc[m:] #données de test, ne pas utiliser qu'à la fin pour tester (calcul du taux) 
testlabels=labels[m:] #sortie désirée (réelle) de chaque donnée de test


## Rapport détaillé :

 j'ai opté pour l'utilisation de la méthode Dtw (Dynamic Time Warping) pour la comparaison des paramètres acoustiques deja extrais par notre methode MFCC. La Déformation Dynamique Temporelle (DTW) permet de mesurer la différence entre deux signaux temporels pouvant présenter des décalages, des déformations l’un par rapport à l’autre. 

L'algorithme Dtw construit une matrice des distances entre les 2 echantillos. Le calcul de la distance enter échantillon sera une distance euclidienne. il faut construire au préalable une matrice des distances , ayant 2 vecteurs X1 et X2 par exemples , on prend chaque point du vecteur X1 et on calcule la distance avec tous les points du vecteur X2, ainsi de suite. chaque ligne (qui représente un segment du signal) de ma matrice est constituée des 13 coéfficients MFCC (en colonne) . Le nombre de ligne de ma matrice représente ainsi le nombre de segments du signal et les colonnes représentent les 13 coéfficints MFCC retenus pour chaque segment, Ma matrice n*13 représente "n" echantillons, et chaque echantillon à 13 parametres. Il nous faut donc créer une distance pour comparer deux vecteurs en dimension 13.

Et tt ce travail sera appliqueé aux audios des 04 locuteurs, c'est pour ca j'ai preferé travaillé avec le trainmfcc et le trainlabels.

In [25]:
from dtw import dtw

In [35]:
#Fonction de reconnaissance :
def reconnaissance(f_mfcc):
    dmin, jmin, i = np.inf, -1, 0
    for x in trainmfcc:   #pour chaque exemple de la base d'apprentissage
        d, _, _, _ = dtw(f_mfcc, x, dist=lambda x, y: np.sum(np.abs(x-y)))   #calculer sa distance avec notre échantillon qu'on veut reconnaitre
        if d < dmin: #si la distance est la plus petite jusqu'à maintenant
            dmin=d  #sauvgarder la distance la plus petite
            jmin=i  #récupérer l'indice de l'exemple le plus proche
       
        i=i+1
        
    return trainlabels[jmin]

In [36]:
###################Test du modèle avec (testmfcc et testlabel)##############################################
reponse=[] #initialiser la liste qui va contenir la réponse de votre méthode pour chaque fichier test
for f_mfcc in testmfcc:
    reponse.append(reconnaissance(f_mfcc))
    
correct=sum(reponse==testlabels) #calcul du nombre de réponses correctes
print('Réponses correctes: %d / %d'%(correct,len(testlabels)))

taux=np.mean(reponse==testlabels) #calcul en pourcentage (taux de reconnaissance)
print('Taux de reconnaissance: %.2f%%'%(taux*100))

Réponses correctes: 976 / 1000
Taux de reconnaissance: 97.60%


#  La reconnaissance du locuteur.

Nous devons etiqueter chaq'un des quatre locuteurs par une etiquete, puis re-faire presque le meme travail que la reconnaissance des chiffres, CAD choisir une méthode (Dtw par exemple) et passer par une partie trainning et une partie test pour pouvoir reconnaitre notre locuteur.